In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from time import sleep
from processor.processor import Processor as p

In [2]:
market = Market()
market.connect()
sp5 = market.retrieve("sp100")
market.disconnect()

In [3]:
start = (datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")

In [4]:
analysis = []
market.connect()
periods = ["year","quarter","week"]
for ticker in tqdm(sp5["ticker"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test = p.column_date_processing(test)
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["rolling_100"] = test["adjopen"].rolling(window=100).mean()
        test["window_10"] = test["adjopen"].shift(10)
        ep = test.tail(1)["adjopen"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjopen"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjopen"].item())/test.tail(2).iloc[0]["adjopen"].item()
        report["window_gain"] = (test.tail(2).iloc[0]["window_10"].item() - ep)/ep
        report["price"] = ep
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:01<00:00, 52.92it/s]


In [18]:
period = "quarter"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["ticker","GICS Sector"]],on="ticker")
a[(a["price"]<25) & (a["price"]>0)].sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,price,ticker,GICS Sector
85,-0.193439,-0.089981,0.017118,0.012065,-0.022440,14.26,T,Communication Services
37,0.075523,-0.211474,-0.018272,-0.015820,0.080372,11.82,F,Consumer Discretionary


In [8]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,price
GICS Sector,,,,,,
Energy,-0.024786,0.043714,-0.025187,-0.018698,-0.017685,126.730000
Health Care,-0.018898,0.040859,0.006990,0.008106,0.003528,211.613571
Industrials,0.059624,0.018333,-0.003372,-0.003379,0.020062,200.607917
Communication Services,0.283951,0.015512,0.012875,0.005154,0.010347,169.957000
Materials,0.142950,-0.002029,-0.003709,-0.007319,0.018328,215.620000
Information Technology,0.365604,-0.008209,0.011154,-0.009215,0.016550,251.799812
Financials,-0.041717,-0.011841,-0.023263,-0.019424,0.068977,146.313333
Utilities,-0.084400,-0.020160,0.003726,0.011988,-0.010979,66.733750
Consumer Staples,-0.016532,-0.024841,-0.004026,0.001720,0.041370,129.746364


In [9]:
a = pd.DataFrame(analysis)
a.sort_values("window_gain",ascending=False).head(10)

,year_gain,quarter_gain,week_gain,day_gain,window_gain,price,ticker
82,-0.316172,0.000532,-0.051898,-0.054439,0.143136,56.45,SCHW
71,-0.177090,-0.126778,-0.080497,-0.039422,0.130439,96.98,NKE
24,0.100560,-0.079346,-0.047687,-0.040945,0.120499,100.25,COF
42,-0.029838,-0.152297,-0.012331,-0.018823,0.104750,32.84,GM
97,-0.265080,-0.047687,0.000754,0.003400,0.102033,26.56,WBA
13,-0.131298,-0.008711,-0.026352,-0.023344,0.100176,28.45,BAC
45,-0.058108,-0.005552,-0.014659,-0.005768,0.096937,320.62,GS
19,-0.073138,-0.098573,-0.036603,-0.029544,0.096444,41.06,C
63,1.348966,0.006278,0.017816,-0.013844,0.089775,288.50,META
29,-0.249833,-0.019542,0.009459,0.003583,0.088651,67.23,CVS
